In [7]:
import numpy as np
import networkx as nx
from itertools import product
import random

def generate_directed_graphs(num_vertices):
    '''
    Генерирует всевозможные ориентированные графы для заданного числа вершин
    '''
    num_edges = num_vertices * (num_vertices - 1)
    
    for edges in product([0, 1], repeat=num_edges):
        adjacency_matrix = np.zeros((num_vertices, num_vertices), dtype=int)
        
        edge_index = 0
        for i in range(num_vertices):
            for j in range(num_vertices):
                if i != j:
                    adjacency_matrix[i, j] = edges[edge_index]
                    edge_index += 1

        yield adjacency_matrix
        
def filter_acyclic_graphs(graph_matrices):
    '''
    Фильтрует графы, исключая те, которые содержат циклы
    '''
    acyclic_graphs = []
    
    for matrix in graph_matrices:
        # Преобразуем матрицу смежности в граф networkx
        G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)
        
        # Проверяем наличие циклов в графе
        if not list(nx.simple_cycles(G)):  # Если список циклов пуст
            acyclic_graphs.append(matrix)
    
    return acyclic_graphs


def filter_isomorphic_graphs(graph_matrices):
    '''
    Исключает изоморфные графы из списка матриц смежности
    '''
    unique_graphs = []
    
    for matrix in graph_matrices:
        # Преобразуем матрицу смежности в граф networkx
        G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)
        
        # Проверяем, есть ли изоморфный граф в списке уникальных графов
        is_isomorphic = False
        for unique_matrix in unique_graphs:
            G_unique = nx.from_numpy_array(unique_matrix, create_using=nx.DiGraph)
            if nx.is_isomorphic(G, G_unique):
                is_isomorphic = True
                break
        
        # Если изоморфных графов нет, добавляем граф в список уникальных
        if not is_isomorphic:
            unique_graphs.append(matrix)
    
    return unique_graphs


def gen_konf(n):
    '''
    Функция генерирует всевозможные варианты раздачи номера машины и времени выполнения работы
    mn_wh[номер работы] = [номер машины, время выполнения]
    '''
    all_konf = []
    for i in range(2**(2*n)):
        all_konf.append('0'*(2*n - len(bin(i)[2:])) + bin(i)[2:])
    
    MN_WH = []    
    for i in all_konf:
        mn_wh = { }
        for j in range(n):
            mn_wh[j+1] = [int(i[2*j]) + 1, int(i[2*j+1]) + 1]
        MN_WH.append(mn_wh)
    return(MN_WH)


def tasks(n):
    '''
    В функции происходит генерирование графов, отсеивание тех, что изоморфны и содержат циклы
    Генерируется подборка всевозможных пар машина-время работы для каждой работы
    Для каждого оставшегося графа происходит генерация словарей для каждого варианта подборки
    
    Функция возвращает массив словарей-заданий
    '''

    MN_WH = gen_konf(n)

    all_graphs = list(generate_directed_graphs(n))
    unique_graphs = filter_acyclic_graphs(filter_isomorphic_graphs(all_graphs))

    TASKS = []
    

    for i, matrix in enumerate(unique_graphs):
        print(f"Unique Graph {i+1}:\n{matrix}\n")
        for mn_wh in MN_WH:
            lib = { }
            for j in range(len(matrix)):
                n=0
                for inp in range(len(matrix[j])):
                    if matrix[j][inp] == 1:
                        n +=1
                        lib[(str(j+1), str(mn_wh.get(j+1)[0]))] = {'dur' : mn_wh.get(j+1)[1], 'prec' : (str(inp+1), str(mn_wh.get(inp+1)[0]))}
                if n == 0:
                    lib[(str(j+1), str(mn_wh.get(j+1)[0]))] = {'dur' : mn_wh.get(j+1)[1], 'prec' : None}
            TASKS.append(lib)
            print(lib)
    return TASKS

TASKS = tasks(int(input('Введите число вершин:')))

Введите число вершин:4
Unique Graph 1:
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]

{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 1, 'prec': None}, ('4', '1'): {'dur': 1, 'prec': None}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 1, 'prec': None}, ('4', '1'): {'dur': 2, 'prec': None}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 1, 'prec': None}, ('4', '2'): {'dur': 1, 'prec': None}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 1, 'prec': None}, ('4', '2'): {'dur': 2, 'prec': None}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 2, 'prec': None}, ('4', '1'): {'dur': 1, 'prec': None}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 2, 'prec': None}, ('4', '1'): {'dur': 2, 'prec': None

Unique Graph 10:
[[0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [1 0 1 0]]

{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 1, 'prec': ('2', '1')}, ('4', '1'): {'dur': 1, 'prec': ('3', '1')}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 1, 'prec': ('2', '1')}, ('4', '1'): {'dur': 2, 'prec': ('3', '1')}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 1, 'prec': ('2', '1')}, ('4', '2'): {'dur': 1, 'prec': ('3', '1')}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 1, 'prec': ('2', '1')}, ('4', '2'): {'dur': 2, 'prec': ('3', '1')}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 2, 'prec': ('2', '1')}, ('4', '1'): {'dur': 1, 'prec': ('3', '1')}}
{('1', '1'): {'dur': 1, 'prec': None}, ('2', '1'): {'dur': 1, 'prec': None}, ('3', '1'): {'dur': 2, 'prec': ('2'

{('1', '2'): {'dur': 1, 'prec': None}, ('2', '2'): {'dur': 1, 'prec': ('1', '2')}, ('3', '1'): {'dur': 2, 'prec': ('2', '2')}, ('4', '1'): {'dur': 1, 'prec': ('3', '1')}}
{('1', '2'): {'dur': 1, 'prec': None}, ('2', '2'): {'dur': 1, 'prec': ('1', '2')}, ('3', '1'): {'dur': 2, 'prec': ('2', '2')}, ('4', '1'): {'dur': 2, 'prec': ('3', '1')}}
{('1', '2'): {'dur': 1, 'prec': None}, ('2', '2'): {'dur': 1, 'prec': ('1', '2')}, ('3', '1'): {'dur': 2, 'prec': ('2', '2')}, ('4', '2'): {'dur': 1, 'prec': ('3', '1')}}
{('1', '2'): {'dur': 1, 'prec': None}, ('2', '2'): {'dur': 1, 'prec': ('1', '2')}, ('3', '1'): {'dur': 2, 'prec': ('2', '2')}, ('4', '2'): {'dur': 2, 'prec': ('3', '1')}}
{('1', '2'): {'dur': 1, 'prec': None}, ('2', '2'): {'dur': 1, 'prec': ('1', '2')}, ('3', '2'): {'dur': 1, 'prec': ('2', '2')}, ('4', '1'): {'dur': 1, 'prec': ('3', '2')}}
{('1', '2'): {'dur': 1, 'prec': None}, ('2', '2'): {'dur': 1, 'prec': ('1', '2')}, ('3', '2'): {'dur': 1, 'prec': ('2', '2')}, ('4', '1'): {'dur'